In [19]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import cv2
from tensorflow.keras.preprocessing.image import array_to_img, ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D

# CSV files

training_data = pd.read_csv("/kaggle/input/facial-keypoints-detection/training.zip")
test_data = pd.read_csv("/kaggle/input/facial-keypoints-detection/test.zip")
id_lookup_table = pd.read_csv("/kaggle/input/facial-keypoints-detection/IdLookupTable.csv")
    
# Data 

def image_to_array(image):
    image = np.array([int(elem) for elem in image.split(" ")])
    image = image.reshape(96,96,1) 
    return image

def get_training_data(training_data=training_data):
    data = training_data.fillna(method = "ffill")
    images = [image_to_array(image) for image in np.array(data["Image"])] 
    x_train, x_test, y_train, y_test = train_test_split(np.array(images, dtype = float), data.drop(["Image"], axis = 1))

    return x_train, x_test, y_train, y_test

def get_testing_data(test_data=test_data):
    images = [image_to_array(image) for image in np.array(test_data["Image"])] 
    return np.array(images, dtype = float)

def create_submission(predictions, id_lookup_table=id_lookup_table):
    features_n = list(id_lookup_table['FeatureName'])
    img_ids = list(id_lookup_table['ImageId']-1)
    row_ids = list(id_lookup_table['RowId'])

    features_lst = [features_n.index(feature) for feature in features_n]

    predictions = [predictions[x][y] for x, y in zip(img_ids, features_lst)]
    submission = pd.DataFrame({
        "RowId":row_ids,
        "Location":predictions
    })
    submission.to_csv("submission.csv",index = False)
    
# Utilities

def show_image(image=X_train[0])
    plt.imshow(image)

In [5]:
x_train, x_test, y_train, y_test = get_training_data()

In [20]:
test_image = get_testing_data()

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),input_shape = (96,96,1),activation = "relu"),
    tf.keras.layers.MaxPooling2D((2,2) ),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(256,(3,3),activation = "relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(512,(3,3),activation = "relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024,activation = "relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512,activation = "relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(30,activation  = "relu")
    
])

model.compile(
    optimizer = "adam",
    loss = "mae",
    metrics = ["accuracy"]
)

model.fit(x_train.astype(float), y_train.astype(float), epochs = 35)


Epoch 1/35
166/166 [==============================] - 173s 1s/step - loss: 16.8804 - accuracy: 0.2507
Epoch 2/35
166/166 [==============================] - 171s 1s/step - loss: 8.5978 - accuracy: 0.3825
Epoch 3/35
166/166 [==============================] - 170s 1s/step - loss: 6.4395 - accuracy: 0.4453
Epoch 4/35
166/166 [==============================] - 168s 1s/step - loss: 4.1144 - accuracy: 0.4707
Epoch 5/35
166/166 [==============================] - 169s 1s/step - loss: 3.8393 - accuracy: 0.4939
Epoch 6/35
166/166 [==============================] - 169s 1s/step - loss: 3.7874 - accuracy: 0.5250
Epoch 7/35
166/166 [==============================] - 169s 1s/step - loss: 3.7071 - accuracy: 0.5193
Epoch 8/35
166/166 [==============================] - 169s 1s/step - loss: 3.6557 - accuracy: 0.5486
Epoch 9/35
166/166 [==============================] - 169s 1s/step - loss: 3.5769 - accuracy: 0.5539
Epoch 10/35
166/166 [==============================] - 169s 1s/step - loss: 3.5965 - accur

In [21]:
predictions = model.predict(test_image)
create_submission(predictions)

56/56 [==============================] - 14s 244ms/step
